In [24]:
import os
import json
import re
import torch
from PIL import Image, ImageDraw, ImageFont
from transformers import AutoTokenizer, AutoModelForCausalLM

# ⚙️ 설정값
INPUT_FILE = "project_full_context.txt"   # 분석할 원본 코드
OUTPUT_JSON = "project_flows.json"        # LLM이 생성할 흐름 데이터
MODEL_ID = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

# GPU 설정 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 실행 환경: {device.upper()}")

🚀 실행 환경: CUDA


In [25]:
print(f"🔄 모델 로딩 시작... ({MODEL_ID})")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

print("✅ 모델 장착 완료!")

🔄 모델 로딩 시작... (Qwen/Qwen2.5-Coder-1.5B-Instruct)
✅ 모델 장착 완료!


In [ ]:
SYSTEM_PROMPT = '''
You are a 'Code Flow Visualization Expert' and Senior Developer.
Analyze the provided code and extract **execution flows** (Call Graph).

### INSTRUCTIONS
1. Identify which function calls which function.
2. Extract the **exact code snippet** where each call happens.
3. Keep descriptions simple for beginners.

### OUTPUT FORMAT (JSON ONLY)
{
  "flows": "signup",
  "bundle": """
app.use("/auth", authRouter);

router.post("/signup", validateSignup, authController.signup);

export async function signup(req, res, next) {
  const { userid, password, name, email, url } = req.body;

  //회원 중복 체크하기
  const found = await authRepository.findByUserid(userid);
  if (found) {
    return res.status(409).json({ message: `${userid}이 이미 있습니다.` });
  }
  const hashed = bcrypt.hashSync(password, config.bcrypt.saltRounds);
  const user = await authRepository.createUser({
    userid,
    password: hashed,
    name,
    email,
    url,
  });

  //   const user = await authRepository.createUser(userid, password, name, email);
  const token = await createJwtToken(user);
  console.log(token);
  res.status(201).json({ token, userid });
}

export async function createUser(user) {
  return new User(user).save().then((data) => data.id);
}"""
}

### RULES
- Output **JSON ONLY**. No markdown.
- The `code_snippet` MUST be exact and exist in the code.
'''

# 청크 단위 프롬프트 (파일별 분석)
CHUNK_PROMPT = """
Analyze this code file and extract function calls.
Focus on: {user_query}

File: {filename}
Code:
{code_snippet}

Return JSON with flows found in this file only.
"""

In [32]:
def parse_files_from_context(code_context):
    """
    project_full_context.txt에서 파일 단위로 분할
    형식: === FILE_PATH: filename ===
    """
    files = {}
    pattern = r"=== FILE_PATH:\s*(.+?)\s*===\n(.*?)(?===\s*FILE_PATH:|$)"
    matches = re.finditer(pattern, code_context, re.DOTALL)
    
    for match in matches:
        filename = match.group(1).strip()
        content = match.group(2).strip()
        files[filename] = content
    
    if not files:
        # 파일 구분이 없으면 전체를 하나로 간주
        files["<full_context>"] = code_context
    
    return files

def analyze_chunk(filename, code_snippet, user_query):
    """
    개별 파일/청크를 LLM으로 분석
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": CHUNK_PROMPT.format(
                filename=filename,
                code_snippet=code_snippet,
                user_query=user_query
            )
        }
    ]

    text_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text_input], return_tensors="pt").to(device)

    print(f"  🔍 분석 중: {filename} ({len(code_snippet)} chars)")

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1028,  # 청크이므로 더 짧게
        temperature=0.1,
        top_p=0.9,
        do_sample=True
    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # 정제
    clean_text = response_text.strip()
    clean_text = re.sub(r"^```(json)?", "", clean_text, flags=re.MULTILINE).strip()
    clean_text = re.sub(r"```$", "", clean_text, flags=re.MULTILINE).strip()

    try:
        result = json.loads(clean_text)
        return result
    except json.JSONDecodeError:
        print(f"    ⚠️ JSON 파싱 실패")
        return {"flows": []}

def analyze_code_flow_chunked(user_query):
    """
    파일 단위 청크 분석 + 통합
    """
    if not os.path.exists(INPUT_FILE):
        print(f"❌ '{INPUT_FILE}' 파일이 없습니다.")
        return False

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        code_context = f.read()

    # 1. 파일 단위로 분할
    files = parse_files_from_context(code_context)
    print(f"📂 발견된 파일: {len(files)}개")

    # 2. 각 파일을 순차적으로 분석
    all_flows = []
    for filename, content in files.items():
        if len(content.strip()) < 50:  # 너무 짧은 파일 스킵
            print(f"  ⏭️  스킵: {filename} (너무 짧음)")
            continue

        result = analyze_chunk(filename, content, user_query)
        flows = result.get("flows", [])
        
        # 파일명 추가
        for flow in flows:
            flow["source_file"] = filename
            all_flows.append(flow)
        
        print(f"    ✅ {len(flows)}개 flow 발견")

    # 3. 통합 JSON 저장
    integrated_data = {"flows": all_flows, "total_flows": len(all_flows)}
    
    with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
        json.dump(integrated_data, f, ensure_ascii=False, indent=2)

    print(f"✅ 통합 JSON 저장: {OUTPUT_JSON}")
    print(f"📊 총 {len(all_flows)}개 flow 추출됨")
    return True

In [ ]:
# 3. 별도 셀: 시각화 생성
def visualize_flow_arrows_enhanced():
    """
    JSON 데이터를 향상된 이미지로 시각화
    """
    if not os.path.exists(OUTPUT_JSON) or not os.path.exists(INPUT_FILE):
        print("❌ 시각화할 데이터 파일이 부족합니다.")
        return None

    try:
        with open(OUTPUT_JSON, 'r', encoding='utf-8') as f:
            data = json.load(f)
            flows = data.get("flows", [])
    except json.JSONDecodeError:
        print("⚠️ JSON 파싱 실패")
        return None

    if not flows:
        print("⚠️ 시각화할 flow가 없습니다.")
        return None

    # 이미지 크기 계산
    img_width = 1400
    flow_height = 150  # 각 flow당 높이
    img_height = max(800, len(flows) * flow_height + 200)
    
    img = Image.new('RGB', (img_width, img_height), color='#1e1e1e')
    draw = ImageDraw.Draw(img)
    
    try:
        font_small = ImageFont.truetype("arial.ttf", 12)
        font_medium = ImageFont.truetype("arial.ttf", 14)
        font_title = ImageFont.truetype("arial.ttf", 18)
    except:
        font_small = ImageFont.load_default()
        font_medium = font_small
        font_title = font_small

    # 제목
    draw.text((50, 30), "📊 Code Flow Visualization", fill="#FFFFFF", font=font_title)
    draw.text((50, 60), f"Total Flows: {len(flows)}", fill="#ABB2BF", font=font_medium)
    
    colors = ["#FF5555", "#50FA7B", "#8BE9FD", "#FFB86C", "#BD93F9", "#FF79C6"]
    y_offset = 120

    print(f"🎨 시각화 생성 중... ({len(flows)} flows)")

    for flow_idx, flow in enumerate(flows):
        flow_color = colors[flow_idx % len(colors)]
        
        # Flow 배경
        draw.rectangle([40, y_offset-10, img_width-40, y_offset+flow_height-20], 
                       outline=flow_color, width=2)
        
        # Flow 제목
        flow_name = flow.get('flow_name', 'Unknown')
        source_file = flow.get('source_file', 'N/A')
        draw.text((60, y_offset+5), f"Flow {flow_idx+1}: {flow_name}", 
                  fill=flow_color, font=font_medium)
        draw.text((60, y_offset+30), f"📁 {source_file}", 
                  fill="#ABB2BF", font=font_small)
        
        sequence = flow.get('sequence', [])
        
        # Sequence 단계별 표시
        step_y = y_offset + 60
        for i, step in enumerate(sequence):
            step_color = colors[(flow_idx + i) % len(colors)]
            
            f_word = step['from'].get('focus_word', '?')
            t_word = step['to'].get('focus_word', '?')
            desc = step.get('description', '')[:60]
            
            # 단계 번호
            draw.text((70, step_y), f"{i+1}.", fill=step_color, font=font_medium)
            
            # 호출 관계
            draw.text((100, step_y), f"{f_word}", fill="#50FA7B", font=font_small)
            draw.text((220, step_y), "→", fill=step_color, font=font_medium)
            draw.text((245, step_y), f"{t_word}", fill="#8BE9FD", font=font_small)
            
            # 설명
            draw.text((400, step_y), desc, fill="#ABB2BF", font=font_small)
            
            step_y += 25
        
        y_offset += flow_height

    return img

# 시각화 실행
print("🎨 시각화 시작...")
result_img = visualize_flow_arrows_enhanced()

if result_img:
    display(result_img)
    result_img.save("final_flow_chart_chunked.png")
    print("✨ 시각화 완료! 저장: final_flow_chart_chunked.png")
else:
    print("❌ 시각화 생성 실패")

In [33]:
# 1. 청크 기반 분석 실행
print("🚀 청크 기반 분석 시작...")
success = analyze_code_flow_chunked(user_query="회원가입(Signup) 시 데이터 이동 흐름")

# 2. 성공 시 JSON 결과 출력
if success:
    with open(OUTPUT_JSON, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print("\n📋 === 추출된 흐름 (JSON) ===\n")
    print(json.dumps(data, ensure_ascii=False, indent=2))
    
    print(f"\n✅ JSON 저장 완료: {OUTPUT_JSON}")
    print(f"📊 총 {data.get('total_flows', 0)}개 flow 발견")
    
    # 각 flow별 요약
    for idx, flow in enumerate(data.get('flows', []), 1):
        print(f"\n[Flow {idx}] {flow.get('flow_name', 'Unknown')}")
        print(f"  파일: {flow.get('source_file', 'Unknown')}")
        sequence = flow.get('sequence', [])
        print(f"  단계: {len(sequence)}개")
        for step in sequence:
            from_word = step['from'].get('focus_word', '?')
            to_word = step['to'].get('focus_word', '?')
            desc = step.get('description', '')
            print(f"    → {from_word} → {to_word}: {desc}")

🚀 청크 기반 분석 시작...
📂 발견된 파일: 1개
  🔍 분석 중: <full_context> (12721 chars)
    ⚠️ JSON 파싱 실패
    ✅ 0개 flow 발견
✅ 통합 JSON 저장: project_flows.json
📊 총 0개 flow 추출됨

📋 === 추출된 흐름 (JSON) ===

{
  "flows": [],
  "total_flows": 0
}

✅ JSON 저장 완료: project_flows.json
📊 총 0개 flow 발견


🧠 AI(3B) 분석 시작... Query: '회원가입(Signup) 시 데이터 이동 흐름'


KeyboardInterrupt: 